# Galassify v2

### Importación de librerías y carga de datos

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import galassify_autoencoder as autoencoder
import galassify_grafica_error, galassify_grafOutlier

from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans, DBSCAN 
from sklearn.metrics import silhouette_samples, silhouette_score

%matplotlib inline
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

from IPython.display import Image
from IPython.core.display import HTML 

En este caso, cargamos directamente los datasets original y el dataset sin negativos, creados en notebooks anteriores. Hay que tener en cuenta que el dataset sin negativos tiene algunos espectros menos que el original y que, por tanto, los índices varían.

In [3]:
data_origen = pd.read_csv("./data/datasetV3_3KRandom.csv", sep=";")
data_origen.head()

,PLATE,MJD,FIBER,Z,3868.645565753641,3869.535542582065,3870.4278489317717,3871.318235772099,3872.2109532069176,3873.1017502480395,3873.992752215851,3874.8860863882537,3875.777498840249,3876.671244571283,3877.563067696571,3878.45722517584,3879.3494591636186,3880.2440285808143,3881.136673620369,3882.029524011743,3882.924711445972,3883.8179731730975,3884.713573019754,3885.6072462721336,3886.503258721214,3887.3973436884376,3888.291634338733,3889.188265801773,3890.082968451347,3890.980012992077,3891.8751278307373,3892.772585639461,3893.668112857104,3894.5638460896052,3895.461923910823,3896.3580698071996,3897.2565613724432,3898.15312012281,3899.052025622691,3899.9489974172493,3900.8461755589897,3901.745702071507,3902.643293542793,3903.543234466747,3904.4412394579995,3905.3415949843097,3906.2400136860347,3907.1386390678003,3908.039616608505,3908.9386559865584,...,7417.524757252301,7419.2311490749735,7420.942007391534,7422.649185348109,7424.356756038729,7426.068796308937,7427.777153676565,7429.489982682914,7431.199127089972,7432.91274519583,7434.622677004906,7436.333002181546,7438.047804147664,7439.758917270303,7441.474509244873,7443.186410676516,7444.902793023495,7446.615483127316,7448.328567233246,7450.0461353501705,7451.760008673134,7453.478368072875,7455.1930309764675,7456.9121820235705,7458.627634871548,7460.3434823572,7462.063821087017,7463.78045906279,7465.501590351842,7467.219019181919,7468.940943395341,7470.659163444065,7472.377778767054,7474.100892579042,7475.820299667295,7477.544207317,7479.264406535282,7480.989108388424,7482.710100101674,7484.431487726802,7486.157381097453,7487.879561765048,7489.606250253956,7491.329224329366,7493.0567083028445,7494.7804761515945,7496.504640550872,7498.233317963899,7499.958276684899,7501.6877504987915
0,2099,53469,148,0.077807,0.636552,0.367806,0.278396,0.285307,0.183786,0.328635,0.450617,0.433929,0.899009,0.643836,0.475315,0.438675,0.467904,0.417998,0.323266,0.554133,0.877506,0.547206,0.296718,0.674742,0.606375,0.222630,0.451282,0.314469,0.190128,0.553508,0.495501,0.428374,1.490322,0.988478,0.554287,0.655043,0.784162,0.499813,0.408123,0.484755,0.473120,0.623950,0.716056,0.306379,0.408779,0.382334,0.384830,0.556806,0.535645,0.374631,...,1.403149,1.300737,0.854908,1.036072,1.284790,1.174821,1.219686,1.320632,1.182485,1.149967,1.135212,1.197466,1.170419,1.177966,1.258235,1.338014,0.992609,1.281974,1.229110,1.421697,1.077822,1.240584,1.259480,1.368396,0.990141,1.229603,1.239413,1.142440,1.154854,1.272670,1.196519,1.218197,1.355392,1.242389,1.247098,1.202379,0.861311,1.392254,1.273615,1.089576,1.155214,1.211078,1.142575,1.233846,1.084954,1.169770,1.161821,1.160405,1.275585,1.238802
1,1815,53884,1,0.091045,0.924821,0.955832,0.954143,0.935017,0.832751,0.854249,0.846123,0.740990,0.776660,0.833628,0.880842,0.866295,0.799464,0.879522,0.877655,0.809836,0.658508,0.459713,0.461437,0.597803,0.624169,0.539646,0.642504,0.783469,0.816040,0.770628,0.738639,0.680130,0.671148,0.666353,0.646091,0.744776,0.752563,0.641500,0.638825,0.777697,0.875334,0.836110,0.780308,0.769351,0.740468,0.693389,0.687770,0.675530,0.765399,0.892660,...,0.779851,0.728282,0.705656,0.796199,0.861313,0.789964,0.813254,0.877838,0.832707,0.821114,0.850069,0.809483,0.786298,0.802253,0.844928,0.827333,0.783233,0.829814,0.783733,0.794661,0.810325,0.747440,0.753007,0.828389,0.864708,0.812884,0.745643,0.751192,0.838403,0.857025,0.785440,0.767616,0.743195,0.724839,0.805945,0.840591,0.810032,0.829138,0.850290,0.763131,0.688105,0.779044,0.817667,0.815495,0.831063,0.807701,0.825111,0.779291,0.775595,0.852922
2,699,52202,92,0.043789,1.258258,1.268783,1.352357,1.341881,1.336860,1.250001,0.957059,0.828157,1.033551,1.245475,1.137744,0.829527,0.732822,0.620484,0.690629,0.910615,0.772459,0.725904,0.839652,0.876705,0.870031,0.748934,0.776170,0.772296,0.753819,0.862681,0.692442,0.552839,0.605622,0.583866,0.573163,0.866969,1.023563,1.088780,1.152049,0.842883,0.891214,1.112096,1.002524,1.031562,1.042673,0.982164,1.165057,0.973574,0.939535,1.107953,...,0.7

In [4]:
data = pd.read_csv("./data/espectros_3k_sin_negativos.csv")
data.head()

,3868.645565753641,3869.535542582065,3870.4278489317717,3871.318235772099,3872.2109532069176,3873.1017502480395,3873.992752215851,3874.8860863882537,3875.777498840249,3876.671244571283,3877.563067696571,3878.45722517584,3879.3494591636186,3880.2440285808143,3881.136673620369,3882.029524011743,3882.924711445972,3883.8179731730975,3884.713573019754,3885.6072462721336,3886.503258721214,3887.3973436884376,3888.291634338733,3889.188265801773,3890.082968451347,3890.980012992077,3891.8751278307373,3892.772585639461,3893.668112857104,3894.5638460896052,3895.461923910823,3896.3580698071996,3897.2565613724432,3898.15312012281,3899.052025622691,3899.9489974172493,3900.8461755589897,3901.745702071507,3902.643293542793,3903.543234466747,3904.4412394579995,3905.3415949843097,3906.2400136860347,3907.1386390678003,3908.039616608505,3908.9386559865584,3909.8400486071882,3910.7395021722573,3911.6413100640398,3912.541178006939,...,7417.524757252301,7419.2311490749735,7420.942007391534,7422.649185348109,7424.356756038729,7426.068796308937,7427.777153676565,7429.489982682914,7431.199127089972,7432.91274519583,7434.622677004906,7436.333002181546,7438.047804147664,7439.758917270303,7441.474509244873,7443.186410676516,7444.902793023495,7446.615483127316,7448.328567233246,7450.0461353501705,7451.760008673134,7453.478368072875,7455.1930309764675,7456.9121820235705,7458.627634871548,7460.3434823572,7462.063821087017,7463.78045906279,7465.501590351842,7467.219019181919,7468.940943395341,7470.659163444065,7472.377778767054,7474.100892579042,7475.820299667295,7477.544207317,7479.264406535282,7480.989108388424,7482.710100101674,7484.431487726802,7486.157381097453,7487.879561765048,7489.606250253956,7491.329224329366,7493.0567083028445,7494.7804761515945,7496.504640550872,7498.233317963899,7499.958276684899,7501.6877504987915
0,0.924821,0.955832,0.954143,0.935017,0.832751,0.854249,0.846123,0.740990,0.776660,0.833628,0.880842,0.866295,0.799464,0.879522,0.877655,0.809836,0.658508,0.459713,0.461437,0.597803,0.624169,0.539646,0.642504,0.783469,0.816040,0.770628,0.738639,0.680130,0.671148,0.666353,0.646091,0.744776,0.752563,0.641500,0.638825,0.777697,0.875334,0.836110,0.780308,0.769351,0.740468,0.693389,0.687770,0.675530,0.765399,0.892660,0.813037,0.865628,0.984975,0.975004,...,0.779851,0.728282,0.705656,0.796199,0.861313,0.789964,0.813254,0.877838,0.832707,0.821114,0.850069,0.809483,0.786298,0.802253,0.844928,0.827333,0.783233,0.829814,0.783733,0.794661,0.810325,0.747440,0.753007,0.828389,0.864708,0.812884,0.745643,0.751192,0.838403,0.857025,0.785440,0.767616,0.743195,0.724839,0.805945,0.840591,0.810032,0.829138,0.850290,0.763131,0.688105,0.779044,0.817667,0.815495,0.831063,0.807701,0.825111,0.779291,0.775595,0.852922
1,1.258258,1.268783,1.352357,1.341881,1.336860,1.250001,0.957059,0.828157,1.033551,1.245475,1.137744,0.829527,0.732822,0.620484,0.690629,0.910615,0.772459,0.725904,0.839652,0.876705,0.870031,0.748934,0.776170,0.772296,0.753819,0.862681,0.692442,0.552839,0.605622,0.583866,0.573163,0.866969,1.023563,1.088780,1.152049,0.842883,0.891214,1.112096,1.002524,1.031562,1.042673,0.982164,1.165057,0.973574,0.939535,1.107953,0.978757,1.160724,1.187305,0.925614,...,0.730175,0.699556,0.673700,0.662072,0.653056,0.636839,0.720679,0.643880,0.542382,0.667748,0.728863,0.591511,0.588671,0.623791,0.604367,0.717579,0.716184,0.728600,0.809191,0.835630,0.755814,0.748969,0.790636,0.735746,0.718552,0.750787,0.777957,0.748585,0.683396,0.681842,0.849851,0.857925,0.773472,0.670337,0.628301,0.721224,0.707150,0.783355,0.727580,0.692765,0.762835,0.709300,0.659228,0.652220,0.759352,0.837513,0.744902,0.661489,0.674855,0.678003
2,1.080315,1.054255,1.186841,1.184086,1.049520,1.030066,1.138974,1.196202,0.992778,0.916529,1.080038,1.004420,0.986589,1.058829,0.970565,0.952958,0.982420,0.909557,0.850695,0.847845,0.961395,0.866690,0.829688,1.085925,1.097143,0.914356,0.840732,0.813613,0.687617,0.757528,0.818865,0.826454,0.893545,0.869621,0.924727,0.973604,0.889258,0.854609,0.949721,0.908795,

In [5]:
data.shape

(2529, 2877)

In [6]:
data_origen.shape

(2920, 2881)

Para luego entrenar los modelos y tener un set de test para las medidas de accuracy, dividimos el dataset en train y test (nos vale un 15% para el test). Recordemos que estamos usando un aprendizaje no supervisado y no tenemos etiquetas.

In [7]:
espectros_train, espectros_test = train_test_split(data, test_size=0.15, random_state=21)

Ahora podemos seguir con el proceso, primero escalando el dataset

In [13]:
#scaler = MinMaxScaler() # = np.array(data.apply(lambda x: (x-x.min()) / (x.max()-x.min())))
#scaler.fit(espectros_train)

# Grabamos el modelo de escalado para predicciones posteriores
#joblib.dump(scaler, "./scaler.save") 

#espectros_train_scaled = pd.DataFrame(scaler.fit_transform(espectros_train), 
#                               columns=espectros_train.columns,
#                               index=espectros_train.index)
#espectros_test_scaled = pd.DataFrame(scaler.transform(espectros_test),
#                             columns=espectros_test.columns,
#                             index=espectros_test.index)

# Esta es la versiós estandarizada de todo el dataset para comparaciones posteriores
#espectros_scaled = pd.DataFrame(scaler.transform(data), 
#                               columns=data.columns,
#                               index=data.index)
#espectros_train_scaled.head()

In [11]:
# En Tensorflow se espera que la entrada venga en formato matriz, no en DataFrame de Pandas.
# recordemos que al perder este formato, perdemos también los nombres de los índices (las frecuencias)


scaler = MinMaxScaler() # = np.array(data.apply(lambda x: (x-x.min()) / (x.max()-x.min())))
#scaler.fit(espectros_train)

# Grabamos el modelo de escalado para predicciones posteriores
#joblib.dump(scaler, "./scaler.save") 

espectros_train_scaled = scaler.fit_transform(espectros_train)
espectros_test_scaled = scaler.transform(espectros_test)

# Esta es la versiós estandarizada de todo el dataset para comparaciones posteriores
espectros_scaled = scaler.transform(data)
espectros_train_scaled

array([[0.04678941, 0.05360419, 0.05564762, ..., 0.41288072, 0.3861915 ,
        0.44435026],
       [0.04287027, 0.043194  , 0.04044057, ..., 0.40744456, 0.36373126,
        0.35944993],
       [0.06481041, 0.06412845, 0.06176416, ..., 0.24853059, 0.21913139,
        0.21226776],
       ...,
       [0.00458402, 0.01198219, 0.01826583, ..., 0.47691685, 0.40893665,
        0.39941599],
       [0.00999957, 0.01665749, 0.01830878, ..., 0.44922983, 0.40203764,
        0.40050846],
       [0.01155025, 0.01636192, 0.00967703, ..., 0.48665918, 0.39929868,
        0.41680726]])

Vamos a estudiar el error que resulta al pasar los espectros por el autoencoder. En estudios previos se ha visto que este error no es el mismo en función de los tramos del espectro, por lo que vamos a dividir el mismo en cuartos y calcular el mse en cada uno de estos cuartos. 

Repetiremos la generación del modelo 10 veces para eliminar en parte el efecto de la aleatoriedad del autoencoder.

In [14]:
repeticiones = 10
mse = []
for i in range(repeticiones):
    print("*** Repetición: ", i)
    encoder_model, autoencoder_model = autoencoder.autoencoder_stacked(entrada=espectros_train_scaled,
                                                  train_set=espectros_train_scaled,
                                                  test_set=espectros_test_scaled)
    espectros_latentes = encoder_model.predict(espectros_scaled)
    espectros_salida = autoencoder_model.predict(espectros_scaled)
    mse_cuartos = []
    for n in range(4):
        num_elems_cuarto = int(len(espectros_scaled) / 4)
        mse_cuartos.append(np.mean(np.mean(np.power(espectros_scaled[(n*num_elems_cuarto):((n+1)*num_elems_cuarto)] -
                                            espectros_salida[(n*num_elems_cuarto):((n+1)*num_elems_cuarto)], 2),
                                   axis=0), axis=0)) 
    mse.append(mse_cuartos)
mse_final = np.mean(mse, axis=0)
print()
print(" --- mse final:", mse_final)

*** Repetición:  0
Train on 2149 samples, validate on 380 samples
Epoch 1/125
2149/2149 [==============================] - 4s 2ms/sample - loss: 0.6987 - accuracy: 4.7002e-04 - val_loss: 0.6868 - val_accuracy: 0.0000e+00
Epoch 2/125
2149/2149 [==============================] - 4s 2ms/sample - loss: 0.6830 - accuracy: 4.8118e-04 - val_loss: 0.6775 - val_accuracy: 0.0000e+00
Epoch 3/125
2149/2149 [==============================] - 4s 2ms/sample - loss: 0.6731 - accuracy: 4.9073e-04 - val_loss: 0.6672 - val_accuracy: 0.0000e+00
Epoch 4/125
2149/2149 [==============================] - 3s 2ms/sample - loss: 0.6638 - accuracy: 4.9121e-04 - val_loss: 0.6596 - val_accuracy: 0.0000e+00
Epoch 5/125
2149/2149 [==============================] - 5s 2ms/sample - loss: 0.6570 - accuracy: 4.9089e-04 - val_loss: 0.6539 - val_accuracy: 0.0000e+00
Epoch 6/125
2149/2149 [==============================] - 4s 2ms/sample - loss: 0.6519 - accuracy: 4.9558e-04 - val_loss: 0.6496 - val_accuracy: 0.0000e+00
Epoc

2149/2149 [==============================] - 4s 2ms/sample - loss: 0.6259 - accuracy: 7.0811e-04 - val_loss: 0.6252 - val_accuracy: 0.0000e+00
Epoch 54/125
2149/2149 [==============================] - 4s 2ms/sample - loss: 0.6259 - accuracy: 7.0795e-04 - val_loss: 0.6251 - val_accuracy: 0.0000e+00
Epoch 55/125
2149/2149 [==============================] - 4s 2ms/sample - loss: 0.6259 - accuracy: 7.0843e-04 - val_loss: 0.6251 - val_accuracy: 0.0000e+00
Epoch 56/125
2149/2149 [==============================] - 5s 2ms/sample - loss: 0.6258 - accuracy: 7.0843e-04 - val_loss: 0.6251 - val_accuracy: 0.0000e+00
Epoch 57/125
2149/2149 [==============================] - 4s 2ms/sample - loss: 0.6258 - accuracy: 7.0859e-04 - val_loss: 0.6250 - val_accuracy: 0.0000e+00
Epoch 58/125
2149/2149 [==============================] - 4s 2ms/sample - loss: 0.6258 - accuracy: 7.1005e-04 - val_loss: 0.6250 - val_accuracy: 0.0000e+00
Epoch 59/125
2149/2149 [==============================] - 5s 2ms/sample - los

2149/2149 [==============================] - 4s 2ms/sample - loss: 0.6252 - accuracy: 7.1215e-04 - val_loss: 0.6242 - val_accuracy: 0.0000e+00
Epoch 106/125
2149/2149 [==============================] - 4s 2ms/sample - loss: 0.6252 - accuracy: 7.1215e-04 - val_loss: 0.6242 - val_accuracy: 0.0000e+00
Epoch 107/125
2149/2149 [==============================] - 4s 2ms/sample - loss: 0.6252 - accuracy: 7.1134e-04 - val_loss: 0.6242 - val_accuracy: 0.0000e+00
Epoch 108/125
2149/2149 [==============================] - 4s 2ms/sample - loss: 0.6252 - accuracy: 7.1167e-04 - val_loss: 0.6242 - val_accuracy: 0.0000e+00
Epoch 109/125
2149/2149 [==============================] - 4s 2ms/sample - loss: 0.6252 - accuracy: 7.1167e-04 - val_loss: 0.6242 - val_accuracy: 0.0000e+00
Epoch 110/125
2149/2149 [==============================] - 4s 2ms/sample - loss: 0.6252 - accuracy: 7.1248e-04 - val_loss: 0.6241 - val_accuracy: 0.0000e+00
Epoch 111/125
2149/2149 [==============================] - 5s 2ms/sample

2149/2149 [==============================] - 4s 2ms/sample - loss: 0.6285 - accuracy: 6.8918e-04 - val_loss: 0.6278 - val_accuracy: 0.0000e+00
Epoch 33/125
2149/2149 [==============================] - 5s 2ms/sample - loss: 0.6282 - accuracy: 6.9064e-04 - val_loss: 0.6276 - val_accuracy: 0.0000e+00
Epoch 34/125
2149/2149 [==============================] - 5s 2ms/sample - loss: 0.6280 - accuracy: 6.9274e-04 - val_loss: 0.6273 - val_accuracy: 0.0000e+00
Epoch 35/125
2149/2149 [==============================] - 5s 2ms/sample - loss: 0.6277 - accuracy: 6.9307e-04 - val_loss: 0.6271 - val_accuracy: 0.0000e+00
Epoch 36/125
2149/2149 [==============================] - 5s 2ms/sample - loss: 0.6276 - accuracy: 6.9468e-04 - val_loss: 0.6269 - val_accuracy: 0.0000e+00
Epoch 37/125
2149/2149 [==============================] - 5s 2ms/sample - loss: 0.6274 - accuracy: 6.9614e-04 - val_loss: 0.6267 - val_accuracy: 0.0000e+00
Epoch 38/125
2149/2149 [==============================] - 5s 2ms/sample - los

2149/2149 [==============================] - 4s 2ms/sample - loss: 0.6254 - accuracy: 7.1474e-04 - val_loss: 0.6244 - val_accuracy: 0.0000e+00
Epoch 85/125
2149/2149 [==============================] - 5s 2ms/sample - loss: 0.6254 - accuracy: 7.1458e-04 - val_loss: 0.6244 - val_accuracy: 0.0000e+00
Epoch 86/125
2149/2149 [==============================] - 5s 2ms/sample - loss: 0.6254 - accuracy: 7.1490e-04 - val_loss: 0.6244 - val_accuracy: 0.0000e+00
Epoch 87/125
2149/2149 [==============================] - 5s 2ms/sample - loss: 0.6253 - accuracy: 7.1425e-04 - val_loss: 0.6244 - val_accuracy: 0.0000e+00
Epoch 88/125
2149/2149 [==============================] - 5s 2ms/sample - loss: 0.6253 - accuracy: 7.1490e-04 - val_loss: 0.6244 - val_accuracy: 0.0000e+00
Epoch 89/125
2149/2149 [==============================] - 4s 2ms/sample - loss: 0.6253 - accuracy: 7.1458e-04 - val_loss: 0.6244 - val_accuracy: 0.0000e+00
Epoch 90/125
2149/2149 [==============================] - 5s 2ms/sample - los

...

2149/2149 [==============================] - 5s 2ms/sample - loss: 0.6268 - accuracy: 7.0131e-04 - val_loss: 0.6261 - val_accuracy: 0.0000e+00
Epoch 42/125
2149/2149 [==============================] - 5s 2ms/sample - loss: 0.6267 - accuracy: 7.0261e-04 - val_loss: 0.6260 - val_accuracy: 0.0000e+00
Epoch 43/125
2149/2149 [==============================] - 5s 2ms/sample - loss: 0.6266 - accuracy: 7.0326e-04 - val_loss: 0.6259 - val_accuracy: 0.0000e+00
Epoch 44/125
2149/2149 [==============================] - 5s 2ms/sample - loss: 0.6265 - accuracy: 7.0358e-04 - val_loss: 0.6258 - val_accuracy: 0.0000e+00
Epoch 45/125
2149/2149 [==============================] - 5s 2ms/sample - loss: 0.6264 - accuracy: 7.0390e-04 - val_loss: 0.6257 - val_accuracy: 0.0000e+00
Epoch 46/125
2149/2149 [==============================] - 5s 2ms/sample - loss: 0.6263 - accuracy: 7.0584e-04 - val_loss: 0.6256 - val_accuracy: 0.0000e+00
Epoch 47/125
2149/2149 [==============================] - 5s 2ms/sample - los

2149/2149 [==============================] - 4s 2ms/sample - loss: 0.6253 - accuracy: 7.1393e-04 - val_loss: 0.6243 - val_accuracy: 0.0000e+00
Epoch 94/125
2149/2149 [==============================] - 4s 2ms/sample - loss: 0.6253 - accuracy: 7.1425e-04 - val_loss: 0.6243 - val_accuracy: 0.0000e+00
Epoch 95/125
2149/2149 [==============================] - 5s 2ms/sample - loss: 0.6253 - accuracy: 7.1425e-04 - val_loss: 0.6243 - val_accuracy: 0.0000e+00
Epoch 96/125
2149/2149 [==============================] - 5s 2ms/sample - loss: 0.6253 - accuracy: 7.1458e-04 - val_loss: 0.6243 - val_accuracy: 0.0000e+00
Epoch 97/125
2149/2149 [==============================] - 3s 2ms/sample - loss: 0.6253 - accuracy: 7.1393e-04 - val_loss: 0.6243 - val_accuracy: 0.0000e+00
Epoch 98/125
2149/2149 [==============================] - 4s 2ms/sample - loss: 0.6253 - accuracy: 7.1361e-04 - val_loss: 0.6243 - val_accuracy: 0.0000e+00
Epoch 99/125
2149/2149 [==============================] - 4s 2ms/sample - los

2149/2149 [==============================] - 4s 2ms/sample - loss: 0.6339 - accuracy: 6.2287e-04 - val_loss: 0.6334 - val_accuracy: 0.0000e+00
Epoch 21/125
2149/2149 [==============================] - 4s 2ms/sample - loss: 0.6333 - accuracy: 6.3193e-04 - val_loss: 0.6327 - val_accuracy: 0.0000e+00
Epoch 22/125
2149/2149 [==============================] - 5s 2ms/sample - loss: 0.6327 - accuracy: 6.4098e-04 - val_loss: 0.6321 - val_accuracy: 0.0000e+00
Epoch 23/125
2149/2149 [==============================] - 5s 2ms/sample - loss: 0.6320 - accuracy: 6.4794e-04 - val_loss: 0.6315 - val_accuracy: 0.0000e+00
Epoch 24/125
2149/2149 [==============================] - 5s 2ms/sample - loss: 0.6315 - accuracy: 6.5376e-04 - val_loss: 0.6309 - val_accuracy: 0.0000e+00
Epoch 25/125
2149/2149 [==============================] - 3s 2ms/sample - loss: 0.6310 - accuracy: 6.6104e-04 - val_loss: 0.6304 - val_accuracy: 0.0000e+00
Epoch 26/125
2149/2149 [==============================] - 5s 2ms/sample - los

2149/2149 [==============================] - 3s 2ms/sample - loss: 0.6255 - accuracy: 7.0908e-04 - val_loss: 0.6246 - val_accuracy: 0.0000e+00
Epoch 73/125
2149/2149 [==============================] - 4s 2ms/sample - loss: 0.6255 - accuracy: 7.0973e-04 - val_loss: 0.6246 - val_accuracy: 0.0000e+00
Epoch 74/125
2149/2149 [==============================] - 4s 2ms/sample - loss: 0.6255 - accuracy: 7.0973e-04 - val_loss: 0.6246 - val_accuracy: 0.0000e+00
Epoch 75/125
2149/2149 [==============================] - 4s 2ms/sample - loss: 0.6255 - accuracy: 7.1070e-04 - val_loss: 0.6246 - val_accuracy: 0.0000e+00
Epoch 76/125
2149/2149 [==============================] - 4s 2ms/sample - loss: 0.6254 - accuracy: 7.1021e-04 - val_loss: 0.6246 - val_accuracy: 0.0000e+00
Epoch 77/125
2149/2149 [==============================] - 3s 2ms/sample - loss: 0.6254 - accuracy: 7.1118e-04 - val_loss: 0.6245 - val_accuracy: 0.0000e+00
Epoch 78/125
2149/2149 [==============================] - 4s 2ms/sample - los

2149/2149 [==============================] - 5s 2ms/sample - loss: 0.6251 - accuracy: 7.1280e-04 - val_loss: 0.6241 - val_accuracy: 0.0000e+00
Epoch 125/125
2149/2149 [==============================] - 4s 2ms/sample - loss: 0.6251 - accuracy: 7.1248e-04 - val_loss: 0.6241 - val_accuracy: 0.0000e+00

 --- mse final: [0.0034111  0.00325158 0.00338689 0.00322353]


Según estos resultados, no parece que haya diferencias notables entre los 4 cuartos.

Ahora vamos a repetir 25 veces el proceso de entrenar el autoencoder, generar los outliers con el procedimiento de máximo error mse y con el procedimiento con DBSCAN.

Después analizaremos cuantas veces se repiten los outliers en esas repeticiones por cada método y cuantos outliers se repiten en ambos métodos.

In [12]:
repeticiones = 25
outliers_mse = []
outliers_dbscan = []
for i in range(repeticiones):
    print("Repetición: ", i)
    encoder_model, autoencoder_model = galassify_autoencoder.autoencoder(entrada=espectros_train_scaled,
                                                  train_set=espectros_train_scaled,
                                                  test_set=espectros_test_scaled)
    espectros_latentes = encoder_model.predict(espectros_scaled)
    espectros_salida = autoencoder_model.predict(espectros_scaled)

    # Outliers metodo 1
    mse = np.mean(np.power(espectros_scaled - espectros_salida, 2), axis=1)
    df_error = pd.DataFrame({'error': mse}, index=espectros_scaled.index)
    print(df_error.index[df_error["error"] > 0.02].tolist())
    outliers_mse.append(df_error.index[df_error["error"] > 0.02].tolist())
    
    # Outliers método 2
    clustering = DBSCAN(eps=3.5, min_samples=2, n_jobs=-1).fit(espectros_latentes)
    labels_DBSCAN = np.unique(clustering.labels_, return_counts=True)
    clusters_DBSCAN = []
    for i in range(-1,len(labels_DBSCAN[0]), 1): # Empezamos en -1 para coger los outliers, en el índice 0 del array labels
        clusters_DBSCAN.append(np.where(clustering.labels_==i))
    print(clusters_DBSCAN[0][0].tolist())
    outliers_dbscan.append(clusters_DBSCAN[0][0].tolist())
    

W1006 19:56:27.999499 140622982924096 deprecation_wrapper.py:119] From /home/joselquin/anaconda3/envs/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1006 19:56:28.000963 140622982924096 deprecation_wrapper.py:119] From /home/joselquin/anaconda3/envs/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1006 19:56:28.003786 140622982924096 deprecation_wrapper.py:119] From /home/joselquin/anaconda3/envs/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1006 19:56:28.023028 140622982924096 deprecation.py:323] From /home/joselquin/anaconda3/envs/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3217: add_dispatch_support.<locals>.wr

Repetición:  0


W1006 19:56:28.463434 140622982924096 deprecation_wrapper.py:119] From /home/joselquin/anaconda3/envs/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 2149 samples, validate on 380 samples
Epoch 1/125
2149/2149 [==============================] - 6s 3ms/step - loss: 0.6995 - acc: 4.6388e-04 - val_loss: 0.6878 - val_acc: 0.0000e+00
Epoch 2/125
2149/2149 [==============================] - 3s 2ms/step - loss: 0.6829 - acc: 4.8151e-04 - val_loss: 0.6759 - val_acc: 0.0000e+00
Epoch 3/125
2149/2149 [==============================] - 3s 2ms/step - loss: 0.6719 - acc: 4.9364e-04 - val_loss: 0.6668 - val_acc: 0.0000e+00
Epoch 4/125
2149/2149 [==============================] - 3s 2ms/step - loss: 0.6636 - acc: 4.9800e-04 - val_loss: 0.6594 - val_acc: 0.0000e+00
Epoch 5/125
2149/2149 [==============================] - 3s 2ms/step - loss: 0.6569 - acc: 5.0334e-04 - val_loss: 0.6537 - val_acc: 0.0000e+00
Epoch 6/125
2149/2149 [==============================] - 3s 2ms/step - loss: 0.6518 - acc: 5.0771e-04 - val_loss: 0.6495 - val_acc: 0.0000e+00
Epoch 7/125
2149/2149 [==============================] - 3s 1ms/step - loss: 0.6482 - acc: 5.09

KeyboardInterrupt: 

In [10]:
def porcentaje_coincidencia(listas):
    union = []
    for i in range(len(listas)):
        union = set(union) | set(listas[i])
    union = list(union)
    repeticion = np.zeros(len(union), dtype=int)
    for ind, item in enumerate(union):
        for i in range(len(listas)):
            if item in listas[i]:
                repeticion[ind] += 1
    return repeticion, union

In [12]:
r_mse, u_mse = porcentaje_coincidencia(outliers_mse)
r_dbscan, u_dbscan = porcentaje_coincidencia(outliers_dbscan)